**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import matplotlib.pyplot as plt

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Dropout,Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

La fonction act retourne une action pour un état s donné. Avec une probabilité $\epsilon$, elle renvoie une action aléatoire prise dans l'espace des actions possibles. Avec une probabilité $1-\epsilon$, elle renvoie l'action correspondant à la valeur de la policy pour l'état s. Cet $\epsilon$ est important car il traduit l'importance entre exploration et exploitation. En effet, plus il est important, et plus on favorisera l'exploration (action aléatoire) à l'exploitation (action prise en accord avec la policy courrante), et inversement.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=75 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Les array position et board sont deux attributs de la classe Environment de taille NxN, et correspondent à l'île sur laquelle se déroule le jeu. L'array board contient les positions des bonus et des malus que le rat peut récupérer. L'array position enregistre la position du rat durant le jeu.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a=np.random.randint(0,4) # on choisit une action au hasard parmi les 4 disponibles
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state=env.reset()
        game_over=False
        win=lose=0
        while not game_over :

            action = agent.learned_act(state) # on récupère l'action correspond à l'état courrant
            state, reward, game_over = env.act(action) #on effectue cette action et on reçoit de l'environnement un reward et le nouvel état dans lequel on se place
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Save as a mp4
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random8.mp4'))

Win/lose count 6.0/12.0. Average score (-6.0)
Win/lose count 12.5/12.0. Average score (-2.75)
Win/lose count 13.5/16.0. Average score (-2.6666666666666665)
Win/lose count 13.0/19.0. Average score (-3.5)
Win/lose count 11.0/13.0. Average score (-3.2)
Win/lose count 7.0/9.0. Average score (-3.0)
Win/lose count 13.5/10.0. Average score (-2.0714285714285716)
Win/lose count 7.5/14.0. Average score (-2.625)
Win/lose count 13.0/15.0. Average score (-2.5555555555555554)
Win/lose count 9.5/10.0. Average score (-2.35)
Win/lose count 6.5/23.0. Average score (-3.6363636363636362)
Win/lose count 9.5/16.0. Average score (-3.875)
Win/lose count 5.0/13.0. Average score (-4.1923076923076925)
Win/lose count 6.5/12.0. Average score (-4.285714285714286)
Win/lose count 14.5/9.0. Average score (-3.6333333333333333)
Final score: -3.6333333333333333


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1)
On a :

$Q^{\pi}(s,a) = E_{p^{\pi}}[\sum\limits_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})\|s_0=s,a_0=a]$

D'où :

$Q^{\pi}(s,a) =E_{p^{\pi}}[\gamma^0r(s,a) + \gamma E_{p^{\pi}}[\sum\limits_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_0=s',a_0=a']$  en réindexant la somme lorsque l'on sort le terme en t=0 et par hypothèse que l'on se place dans un Markov Decision Process

D'où :

$Q^{\pi}(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$

2)
On a donc :   

$Q^{\pi}(s,a)= r(s,a) + \gamma\sum\limits_{s'\in S}p(s'|s,a)V^{\pi}(s')$
avec $V^{\pi}(s)=Q(s,\pi(s))$

Et, la policy optimale $\pi^*$, sous réserve d'existence, satisfait la relation suivante :

$\pi^*(s)=\underset{a}{argmax}(Q^*(s,a))$

D'où :

$V^*(s)=Q(s,\pi^*(s))=\underset{a \in A}{max}(Q(s,a))$

On a alors :

$Q^{*}(s,a)= r(s,a) + \gamma\sum\limits_{s'\in S}p(s'|s,a)\underset{a'}{max}Q^{*}(s',a')$

On retrouve donc bien :

$Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\underset{a'}{max}Q^{*}(s',a')]$


3)
On cherche donc à trouver $\theta$ tel que $Q(\theta)$ satisfait pour tout s et a :

$Q(s,a,\theta)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\underset{a'}{max}Q(s',a',\theta)]$

Pour cela, nous pouvons donc chercher à minimiser la distance entre les membres de chaque côté de l'égalité. On peut donc considérer comme fonction de perte cette même distance. Or, pour avoir une fonction de perte différentiable, nous devons considérer la norme au carré.

On peut donc chercher à minimiser la fonction de perte suivante pour trouver le paramètre $\theta$ tel que la fonction $Q(\theta)$ soit optimale :

$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>self.max_memory: # on teste si la mémoire est pleine et on attribut à une case aléatoire la valeur m si c'est le cas
          self.memory[np.random.randint(len(self.memory))]=m
        else:
          self.memory.append(m)
    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
         return np.argmax(self.model.predict(np.expand_dims(s,axis=0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
           
            s_i,n_s_i,a_i,r_i,game_over_i = self.memory.random_access() # on récupère une transition aléatoire
            input_states[i]=s_i #on sauvegarde pour chaque transition l'état initial
            target_q[i,:]=self.model.predict(np.expand_dims(s_i,axis=0)) #on remplit le vecteur cible pour toutes les actions possibles avec la prediction du réseau.
            #Nous aurions également pu laisser ces valeurs à 0, mais cela peut provoquer des conflits avec le fonction de loss ou le calcul du gradient


            if not game_over_i:#pour l'action qui a été choisie lors de la transition, on met à jour avec la relation suivie par Q* 
              target_q[i,a_i]=r_i+self.discount*np.max(self.model.predict(np.expand_dims(n_s_i,axis=0)))
            else:#si le jeu est terminé, il n'y pas d'état suivant et on précise donc juste le reward obtenu
              target_q[i,a_i] = r_i
              
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):

    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        ''' J'ai utilisé un modèle simple de couches entièrement connectées, avec 1 couche à 64 neurones et une couche à 16 neurones,
        avec de la batch normalization et du dropout pour faciliter la généralisation du modèle sur la phase de test'''
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(64))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Activation('relu'))
        model.add(Dense(16))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Activation('relu'))
        model.add(Dense(4))

        model.compile(adam(lr=lr, decay=1e-4), "mse") #J'ai utilisé l'optimizer Adam avec lequel j'ai obtenu de meilleurs résultats que sgd
        self.model = model
        

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.4)
agent = DQN_FC(size, lr=0.001, epsilon = 0.15, memory_size=2500, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train70.mp4'))

Epoch 000/075 | Loss 0.5868 | Win/lose count 7.0/3.0 (4.0)
Epoch 001/075 | Loss 0.2023 | Win/lose count 8.5/5.0 (3.5)
Epoch 002/075 | Loss 0.0675 | Win/lose count 6.0/17.0 (-11.0)
Epoch 003/075 | Loss 0.0560 | Win/lose count 6.5/10.0 (-3.5)
Epoch 004/075 | Loss 0.0211 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/075 | Loss 0.0254 | Win/lose count 5.0/9.0 (-4.0)
Epoch 006/075 | Loss 0.0154 | Win/lose count 4.0/3.0 (1.0)
Epoch 007/075 | Loss 0.0075 | Win/lose count 8.0/12.0 (-4.0)
Epoch 008/075 | Loss 0.0126 | Win/lose count 3.5/8.0 (-4.5)
Epoch 009/075 | Loss 0.0147 | Win/lose count 1.5/5.0 (-3.5)
Epoch 010/075 | Loss 0.0175 | Win/lose count 5.0/3.0 (2.0)
Epoch 011/075 | Loss 0.0036 | Win/lose count 5.5/3.0 (2.5)
Epoch 012/075 | Loss 0.0167 | Win/lose count 4.5/9.0 (-4.5)
Epoch 013/075 | Loss 0.0130 | Win/lose count 7.0/6.0 (1.0)
Epoch 014/075 | Loss 0.0203 | Win/lose count 7.0/12.0 (-5.0)
Epoch 015/075 | Loss 0.0068 | Win/lose count 6.0/5.0 (1.0)
Epoch 016/075 | Loss 0.0105 | Win/lose coun

On observe qu'en utilisant un learning rate plus faible, on a de meilleurs performances. Plus espilon est élevé, et plus l'agent pourra explorer son environnement mais donc, plus on aura du mal à converger vers un score élevé car l'agent aura plus tendance à osciller autour de la même position. La taille de la mémoire permet d'obtenir de meilleurs performances mais augmente considérablement les temps de calculs. Enfin, je n'ai pas observé d'influence positive en augmentant la taille de batch.

Pour le meilleur compromis entre ces différents paramètres, j'ai obtenu à l'issu de l'apprentissage un score de 11 au bout de 75 époques. J'ai choisi d'utiliser un nombre d'époques assez élevés car des réseaux entièrement connectés sont assez rapides à entraîner et je souhaitais obtenir les meilleurs performances possibles.

Le score obtenu est satisfaisant. Néanmoins, le nombre d'époques nécessaire pour obtenir ce score est assez élevé. Un réseau de couches entièrement connectées n'est peut être alors pas suffisant pour retrouver $Q^*$.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        #J'ai choisi pour ce réseau convolutif une structure à base de couches de convolutions avec 16 filtres
        model = Sequential()
        model.add(Conv2D(16,3,input_shape=(5,5,self.n_state,),))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Activation('relu'))
        model.add(Conv2D(16,3))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(adam(lr=lr, decay=1e-4), "mse") #J'ai choisi ici aussi d'utiliser l'optimizer Adam
        self.model = model

In [20]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.008, epsilon = 0.15, memory_size=2500, batch_size = 32)
train(agent,env,50,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0151 | Win/lose count 8.5/6.0 (2.5)
Epoch 001/050 | Loss 0.0090 | Win/lose count 1.0/3.0 (-2.0)
Epoch 002/050 | Loss 0.0068 | Win/lose count 4.0/5.0 (-1.0)
Epoch 003/050 | Loss 0.0069 | Win/lose count 6.0/1.0 (5.0)
Epoch 004/050 | Loss 0.0163 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/050 | Loss 0.0105 | Win/lose count 1.0/9.0 (-8.0)
Epoch 006/050 | Loss 0.0047 | Win/lose count 3.0/4.0 (-1.0)
Epoch 007/050 | Loss 0.0175 | Win/lose count 5.5/5.0 (0.5)
Epoch 008/050 | Loss 0.0080 | Win/lose count 5.0/3.0 (2.0)
Epoch 009/050 | Loss 0.0066 | Win/lose count 3.5/7.0 (-3.5)
Epoch 010/050 | Loss 0.0082 | Win/lose count 4.5/2.0 (2.5)
Epoch 011/050 | Loss 0.0068 | Win/lose count 3.5/3.0 (0.5)
Epoch 012/050 | Loss 0.0012 | Win/lose count 5.5/2.0 (3.5)
Epoch 013/050 | Loss 0.0105 | Win/lose count 3.0/2.0 (1.0)
Epoch 014/050 | Loss 0.0030 | Win/lose count 1.0/2.0 (-1.0)
Epoch 015/050 | Loss 0.0143 | Win/lose count 7.5/8.0 (-0.5)
Epoch 016/050 | Loss 0.0045 | Win/lose count 4.5/

On observe ici une influence similaire des hyperparamètres. J'ai néanmoins été contraint de diminuer le nombre d'époques car les temps de calculs étaient trop longs pour tester plusieurs combinaisons. En augmentant, le learning rate, je suis parvenu à obtenir un score de 8 en seulement 50 époques, avec peu d'itérations avec un score négatif et des pointes jusqu'à 15. L'utilisation d'un CNN semble donc être préférable à un reseau FC. Nous allons maintenant observer les performances de ces modèles sur le jeu de test pour comparer leur efficacité.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [21]:
env = Environment(grid_size=size, max_time=T,temperature=0.4)
agent_cnn = DQN_CNN(size, lr=0.008, epsilon = 0.15, memory_size=2500, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=0.001, epsilon = 0.15, memory_size=2500, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/3.0. Average score (-1.0)
Win/lose count 14.0/5.0. Average score (4.0)
Win/lose count 14.0/2.0. Average score (6.666666666666667)
Win/lose count 7.0/2.0. Average score (6.25)
Win/lose count 26.0/1.0. Average score (10.0)
Win/lose count 28.5/10.0. Average score (11.416666666666666)
Win/lose count 23.5/5.0. Average score (12.428571428571429)
Win/lose count 20.0/4.0. Average score (12.875)
Win/lose count 11.5/5.0. Average score (12.166666666666666)
Win/lose count 21.0/6.0. Average score (12.45)
Final score: 12.45
Test of the FC
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0.5/0. Average score (0.16666666666666666)
Win/lose count 2.5/0. Average score (0.75)
Win/lose count 4.0/0. Average score (1.4)
Win/lose count 1.0/0. Average score (1.3333333333333333)
Win/lose count 1.5/0. Average score (1.3571428571428572)
Win/lose count 1.5/0. Average score (1.375)
Win/lose count 0.5/0. Average score (1.27777777777777

In [22]:
HTML(display_videos('cnn_test9.mp4'))

In [23]:
HTML(display_videos('fc_test9.mp4'))

On observe ici une mauvaise généralisation du modèle FC. En effet, on a un score de seulement 1.25 en moyenne. Cependant, que les performances du CNN sont même meilleures avec un score de 12.45, ce qui confirme notre première intuition de meilleur performance du réseau CNN.

Pour différents niveaux de températures, on remarque une baisse de score pour les deux réseaux quand on diminue la température à 0.2. En effet, il y a moins de reward et donc il est plus difficile de les accumuler. En revanche, lorsque l'on augmente la température, le score avec le CNN augmente (on trouve en moyenne un score de 1 contre 0.7 avec une température de 0.3), alors que l'on obtient que des faibles scores pour le réseau FC. J'ai donc choisi de conserver cette température.

Enfin, on observe sur les deux simulations que l'agent commence par explorer puis finit rapidement par osciller autour de la même position. Cela peut s'expliquer par le fait que nous n'ayons pas favorisé l'exploration et que donc l'agent va préférer osciller autour de la même position pour ne pas risquer d'accumuler de nouveaux malus, ce qui implique des scores faibles. Nous allons donc maintenant essayer de favoriser l'exploration pour obtenir de meilleurs scores.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,decay=0.5,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.set_epsilon(agent.epsilon*decay)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1,malus=0.3):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.malus = malus
        #board on which one plays
        self.malus_position=np.zeros((grid_size,grid_size)) #on initialise les malus
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1

        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y] #on retire le malus si l'agent est déjà passé par cette case
        self.malus_position[self.x, self.y] = self.malus
        reward += self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        self.malus_position=np.zeros((self.grid_size,self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [40]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.4,malus=0.1)
agent = DQN_CNN(size, lr=0.008, epsilon = 0.3, memory_size=2500, batch_size = 32,n_state=3)
train_explore(agent, env, 18, decay=0.95,prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/018 | Loss 0.0240 | Win/lose count 6.0/25.100000000000087 (-19.100000000000087)
Epoch 001/018 | Loss 0.0160 | Win/lose count 6.5/20.300000000000026 (-13.800000000000026)
Epoch 002/018 | Loss 0.0113 | Win/lose count 5.0/17.69999999999998 (-12.699999999999982)
Epoch 003/018 | Loss 0.0040 | Win/lose count 2.5/21.50000000000003 (-19.00000000000003)
Epoch 004/018 | Loss 0.0185 | Win/lose count 5.5/21.20000000000003 (-15.700000000000031)
Epoch 005/018 | Loss 0.0070 | Win/lose count 9.5/25.300000000000033 (-15.800000000000033)
Epoch 006/018 | Loss 0.0117 | Win/lose count 13.5/27.500000000000117 (-14.000000000000117)
Epoch 007/018 | Loss 0.0060 | Win/lose count 12.5/24.300000000000043 (-11.800000000000043)
Epoch 008/018 | Loss 0.0121 | Win/lose count 15.0/19.600000000000026 (-4.600000000000026)
Epoch 009/018 | Loss 0.0202 | Win/lose count 21.5/18.900000000000006 (2.5999999999999943)
Epoch 010/018 | Loss 0.0130 | Win/lose count 8.5/19.40000000000001 (-10.90000000000001)
Epoch 011/018 

In [41]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 17.0/2.0. Average score (15.0)
Win/lose count 7.0/4.0. Average score (9.0)
Win/lose count 11.0/1.0. Average score (9.333333333333334)
Win/lose count 21.0/3.0. Average score (11.5)
Win/lose count 13.0/1.0. Average score (11.6)
Win/lose count 14.5/2.0. Average score (11.75)
Win/lose count 16.5/0. Average score (12.428571428571429)
Win/lose count 11.5/1.0. Average score (12.1875)
Win/lose count 17.5/1.0. Average score (12.666666666666666)
Win/lose count 22.0/0. Average score (13.6)
Final score: 13.6


On observe que l'on obtient un meilleur score, avec très peu de malus en test et d'époques en apprentissage (18), ce qui valide la petinence de la démarche proposée.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***